In [ ]:
%load_ext autoreload
%autoreload 1
# %matplotlib widget
import matplotlib
import librosa
import numpy as np
matplotlib.use('QT5Agg')

In [ ]:
def process_test_dict(loaded_dict, row_indices=[0, 4], col_size=512, output_format="channels_first"):
    processed_data = {}

    for key, (test_x, test_y, fund_freq_lst, distances_lst, file_names_lst, orig_signal_lst) in loaded_dict.items():
        test_x = np.array(test_x)
        test_y = np.array(test_y)
        distances_lst = np.array(distances_lst)
        fund_freq_lst = np.array(fund_freq_lst)
        orig_signal_lst = np.array(orig_signal_lst)

        # Select rows and columns size (preserve original order - no sorting)
        X = test_x[:, row_indices, :col_size]
        y = test_y

        # Reshape based on desired output format
        if output_format == "channels_first":
            X = X[:, :, np.newaxis, :]  # [N, num_channels, 1, col_size]
        elif output_format == "channels_last":

            X = X[:, np.newaxis, :, :]  # [N, 1, num_channels, col_size]
        else:
            raise ValueError("output_format must be 'channels_first' or 'channels_last'")

        # Return all data in original order (no train/val split)
        processed_data[key] = (X, y, fund_freq_lst, distances_lst, file_names_lst, orig_signal_lst)

    return processed_data

In [ ]:
import os
print(os.getcwd())

In [ ]:
# """
# testing only ppsp
# """
# import warnings, pickle
# import os
# warnings.filterwarnings("ignore")
# mode = "block"
# # all_combs_lists = [[0], [3], [0, 1, 2, 3]]
# all_combs_lists = [[0]]
# import torch
# from models import fpn_2
# import matplotlib.pyplot as plt
#
# device = 'cpu'
#
#
# with open(f"../conv2d_data/conv2d_psd_scaled_down_1up_{mode}_test.pkl", "rb") as f:
#     loaded_dict_test = pickle.load(f)
#
# for ind, comb_lst in enumerate(all_combs_lists):
#
#     dir_path = f"./conv2d_data/pred_plots/{comb_lst}/"
#     os.makedirs(dir_path, exist_ok=True)
#
#     trained_model = fpn_2.PPSP(in_channels=len(comb_lst),out_channels=32)
#     model_name = f"../best_fpn2_1up_model_{mode}_{comb_lst}.pth"
#     trained_model.load_state_dict(torch.load(f"{model_name}", map_location=torch.device('cpu')))
#     trained_model.eval()
#
#     processed_test = process_test_dict(
#         loaded_dict_test,
#         row_indices=comb_lst,
#         col_size=1024,
#         output_format="channels_last"  # or "channels_first"
#     )
#     # device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
#     print(f"Using device: {device}")
#
#     for key, (X_test, y_test, fund_freq, distances, file_names, orig_sig) in processed_test.items():
#         print(f"Key: {key}")
#         prediction_lst = []
#         freq_prediction_lst, fund_freq_lst = [], []
#         if key == key:
#             for ind in range(len(distances)):
#                 if ind == ind:
#                     cur_fund_freq = fund_freq[ind]
#                     cur_fil_name = file_names[ind]
#
#                     # cur_x = np.array(X_test[ind])[np.newaxis,:,:,:]
#                     cur_x = np.array(X_test[ind])[:, :, :]
#                     torch_x = torch.FloatTensor(cur_x).to('cpu')
#                     cur_y = np.array(y_test[ind])
#
#                     cur_prediction = trained_model(torch_x)
#
#                     fund_pred = torch.sigmoid(cur_prediction).squeeze(1).squeeze(0).detach().cpu().numpy()
#                     # all_harmonic_pred1 = torch.sigmoid(cur_prediction[1]).squeeze(1).squeeze(0).detach().cpu().numpy()
#
#
#                     cur_fund_prediction_norm = (fund_pred - np.min(fund_pred)) / (np.max(fund_pred) - np.min(fund_pred) + 1e-12)
#                     # cur_harmonic_prediction_norm1 = (all_harmonic_pred1 - np.min(all_harmonic_pred1)) / (np.max(all_harmonic_pred1) - np.min(all_harmonic_pred1) + 1e-12)
#
#
#                     binary_cur_truth=cur_y
#
#
#                     plt_fil_name = f"{cur_fund_freq}-{cur_fil_name}_{comb_lst}"
#
#                     plt.title(f"{cur_fund_freq} - {cur_fil_name}_{comb_lst}")
#                     for ind_x in range(cur_x.squeeze(0).shape[0]):
#                         plt.plot(cur_x.squeeze(0)[ind_x],linewidth=1.2)
#
#                     plt.plot(binary_cur_truth, linewidth=1.1, label="True",alpha=0.8)
#
#
#                     plt.plot(cur_fund_prediction_norm, linewidth=0.8, label=" f0",alpha=0.8)
#                     # plt.plot(cur_harmonic_prediction_norm1, '--', linewidth=0.7, label="raw all_harmonics",alpha=0.8)
#
#                     plt.legend(loc='lower right')
#
#                     # plt.savefig(os.path.join(dir_path, f"{plt_fil_name}.png"), dpi=150)
#                     # plt.close()
#
#                     plt.show()

In [ ]:
"""
testing only ppsp_1up head
"""
import warnings, pickle
import os
warnings.filterwarnings("ignore")
mode = "block"
# all_combs_lists = [[0], [3], [0, 1, 2, 3]]
all_combs_lists = [[0],[3]]
import torch
from models import fpn_2
import matplotlib.pyplot as plt
from models import ppsp_1up_head

device = 'cpu'


with open(f"../conv2d_data/conv2d_psd_scaled_down_1up_{mode}_test.pkl", "rb") as f:
    loaded_dict_test = pickle.load(f)

for ind, comb_lst in enumerate(all_combs_lists):

    dir_path = f"./conv2d_data/pred_plots/{comb_lst}/"
    os.makedirs(dir_path, exist_ok=True)

    ## load pretrained ppsp
    trained_model = fpn_2.PPSP(in_channels=len(comb_lst),out_channels=32)
    model_name = f"./model_weights/ppsp_weights.pth"
    trained_model.load_state_dict(torch.load(f"{model_name}", map_location=torch.device('cpu')))

    ### load the ppsp_1up head
    ppsp_1up = ppsp_1up_head.PPSP_withFundamental(pretrained_ppsp=trained_model, freeze=True, hidden=256)
    ppsp1up_model_name=f"../best_fpn2_1up_model_{mode}_{comb_lst}.pth"
    ppsp_1up.load_state_dict((torch.load(f"{ppsp1up_model_name}", map_location=torch.device('cpu'))))
    ppsp_1up.eval()

    processed_test = process_test_dict(
        loaded_dict_test,
        row_indices=comb_lst,
        col_size=1024,
        output_format="channels_last"  # or "channels_first"
    )
    # device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    for key, (X_test, y_test, fund_freq, distances, file_names, orig_sig) in processed_test.items():
        print(f"Key: {key}")
        prediction_lst = []
        freq_prediction_lst, fund_freq_lst = [], []
        if key == key:
            for ind in range(len(distances)):
                if ind == ind:
                    cur_fund_freq = fund_freq[ind]
                    cur_fil_name = file_names[ind]

                    # cur_x = np.array(X_test[ind])[np.newaxis,:,:,:]
                    cur_x = np.array(X_test[ind])[:, :, :]
                    torch_x = torch.FloatTensor(cur_x).to('cpu')
                    cur_y = np.array(y_test[ind])

                    cur_prediction = ppsp_1up(torch_x)

                    fund_pred = torch.sigmoid(cur_prediction[0]).squeeze(1).squeeze(0).detach().cpu().numpy()
                    harmonic_pred = torch.sigmoid(cur_prediction[1]).squeeze(1).squeeze(0).detach().cpu().numpy()
                    # all_harmonic_pred1 = torch.sigmoid(cur_prediction[1]).squeeze(1).squeeze(0).detach().cpu().numpy()


                    cur_fund_prediction_norm = (fund_pred - np.min(fund_pred)) / (np.max(fund_pred) - np.min(fund_pred) + 1e-12)
                    # cur_harmonic_prediction_norm1 = (all_harmonic_pred1 - np.min(all_harmonic_pred1)) / (np.max(all_harmonic_pred1) - np.min(all_harmonic_pred1) + 1e-12)


                    binary_cur_truth=cur_y


                    plt_fil_name = f"{cur_fund_freq}-{cur_fil_name}_{comb_lst}"

                    plt.title(f"{cur_fund_freq} - {cur_fil_name}_{comb_lst}")
                    for ind_x in range(cur_x.squeeze(0).shape[0]):
                        plt.plot(cur_x.squeeze(0)[ind_x],linewidth=1.2)

                    plt.plot(binary_cur_truth, linewidth=1.1, label="True",alpha=0.8)


                    plt.plot(cur_fund_prediction_norm, linewidth=0.8, label=" f0",alpha=0.8)
                    plt.plot(harmonic_pred, '--', linewidth=0.8, label="raw all_harmonics",alpha=0.8)

                    plt.legend(loc='lower right')

                    plt.savefig(os.path.join(dir_path, f"{plt_fil_name}.png"), dpi=150)
                    plt.close()

                    # plt.show()